In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName("News Flash").getOrCreate()

In [24]:
news_flash = spark.read.option("header", True).csv('market-basket.csv', sep = ';')

In [25]:
news_flash.printSchema()

root
 |-- BillNo: string (nullable = true)
 |-- Itemname: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [33]:
from pyspark.sql import functions as F

In [32]:
basketcase = news_flash.dropDuplicates(["BillNo", "Itemname"]).sort("BillNo")

In [34]:
bask = basketcase.groupby("BillNo").agg(F.collect_list("Itemname")).sort("BillNo")

In [35]:
bask.show()

+------+----------------------+
|BillNo|collect_list(Itemname)|
+------+----------------------+
|536365|  [KNITTED UNION FL...|
|536366|  [HAND WARMER UNIO...|
|536367|  [BOX OF VINTAGE J...|
|536368|  [YELLOW COAT RACK...|
|536369|  [BATH BUILDING BL...|
|536370|  [SPACEBOY LUNCH B...|
|536371|  [PAPER CHAIN KIT ...|
|536372|  [HAND WARMER UNIO...|
|536373|  [GLASS STAR FROST...|
|536374|  [VICTORIAN SEWING...|
|536375|  [SAVE THE PLANET ...|
|536376|  [RED HANGING HEAR...|
|536377|  [HAND WARMER RED ...|
|536378|  [PACK OF 60 PINK ...|
|536380|  [JAM MAKING SET P...|
|536381|  [ZINC WILLIE WINK...|
|536382|  [VINTAGE SNAKES &...|
|536384|  [ENAMEL BREAD BIN...|
|536385|  [TRADITIONAL CHRI...|
|536386|  [JUMBO BAG RED RE...|
+------+----------------------+
only showing top 20 rows



In [16]:
from pyspark.ml.fpm import FPGrowth

In [44]:
growth = FPGrowth(itemsCol = "collect_list(Itemname)", minSupport = 0.0005, minConfidence = 0.0006)

In [45]:
model = growth.fit(bask)

In [46]:
model.freqItemsets.show()

Py4JJavaError: An error occurred while calling o267.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 59.0 failed 1 times, most recent failure: Lost task 0.0 in stage 59.0 (TID 30) (10.21.92.160 executor driver): java.lang.StackOverflowError
	at java.base/java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1429)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1179)
	at java.base/java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1553)
	at java.base/java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1510)
	at java.base/java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1433)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1179)
	at java.base/java.io.ObjectOutputStream.writeObject(ObjectOutputStream.java:349)
	at scala.collection.mutable.HashMap.$anonfun$writeObject$1(HashMap.scala:190)
	at scala.coll